In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config('spark.shuffle.useOldFetchProtocol', 'true'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

#### Loading loan_repayments

In [2]:
loan_repayments_raw_df = spark.read \
.format("csv") \
.option("header",True) \
.option("inferSchema", True) \
.load("/user/itv018355/lendingclubproject/raw/loan_repayments_csv")

In [3]:
loan_repayments_raw_df

+-------+---------------+-------------+------------------+------------------+---------------+------------+------------+
|loan_id|total_rec_prncp|total_rec_int|total_rec_late_fee|       total_pymnt|last_pymnt_amnt|last_pymnt_d|next_pymnt_d|
+-------+---------------+-------------+------------------+------------------+---------------+------------+------------+
|6300587|        12000.0|      1864.87|20.860000009100002|13885.726104168401|         108.83|    Nov-2014|        null|
|6311418|         7125.0|        95.92|               0.0| 7220.924234675899|        7220.52|    Sep-2013|        null|
|6321535|        24000.0|      7418.84|               0.0|          31418.84|       13979.48|    Apr-2016|        null|
|6300526|        24000.0|      2296.28|               0.0|  26296.2757411036|         730.17|    Aug-2016|        null|
|6301469|        18000.0|      9239.79|               0.0|   27239.794818997|         452.24|    Aug-2018|        null|
|6306881|        20000.0|      2469.32|               0.0|  22469.3219387879|       16744.64|    Oct-2014|        null|
|6301579|        12000.0|      2089.07|               0.0|          14089.07|        4189.07|    Oct-2015|        null|
|6291661|        28000.0|     12441.42|               0.0|  40441.4227009823|       10660.91|    Apr-2017|        null|
|6311612|         3629.9|       847.39|               0.0|           4478.25|         179.13|    Sep-2015|        null|
|6311704|        23600.0|       2077.3|               0.0|   25677.297703591|       21445.23|    Apr-2014|        null|
|6301559|        13375.0|      3926.19|               0.0|  17301.1873419706|         480.18|    Aug-2016|        null|
|6301412|        17500.0|      2062.85|               0.0|19562.849989482198|        3748.28|    Jan-2016|        null|
|6331614|        14000.0|      2442.47|               0.0|  16442.4700020926|        4430.47|    Nov-2015|        null|
|6331589|        1458.43|      1100.69|               0.0|           4051.94|         426.57|    Mar-2014|        null|
|6321730|        14000.0|       719.51|               0.0|14719.505688732199|       11239.57|    May-2014|        null|
|6321586|        15000.0|      1929.76|   25.290000000028|  16955.0514482354|       10354.66|    Oct-2014|        null|
|6331533|        10000.0|      2248.02|               0.0|  12248.0212664276|         339.97|    Aug-2016|        null|
|6321666|        6239.23|      1316.72|               0.0|           7819.45|         260.55|    Jan-2016|        null|
|6328050|        10000.0|      2617.64|               0.0|  12617.6399999982|        4937.87|    Jun-2015|        null|
|6305315|         6000.0|       515.17|               0.0|           6515.17|        4733.17|    Jun-2014|        null|
+-------+---------------+-------------+------------------+------------------+---------------+------------+------------+
only showing top 20 rows

#### See the schema

In [4]:
loan_repayments_raw_df.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- total_rec_prncp: string (nullable = true)
 |-- total_rec_int: string (nullable = true)
 |-- total_rec_late_fee: string (nullable = true)
 |-- total_pymnt: string (nullable = true)
 |-- last_pymnt_amnt: string (nullable = true)
 |-- last_pymnt_d: string (nullable = true)
 |-- next_pymnt_d: string (nullable = true)



#### Defining schema

In [5]:
loan_repayments_schema = 'loan_id string, total_principal_received float, total_interest_received float, total_late_fee_received float, total_payment_received float, last_payment_amount float, last_payment_date string, next_payment_date string '

#### creating a dataframe using this schema

In [6]:
loan_repayments_raw_df = spark.read \
.format("csv") \
.option("header", True) \
.schema(loan_repayments_schema) \
.load("/user/itv018355/lendingclubproject/raw/loan_repayments_csv")

In [7]:
loan_repayments_raw_df.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- total_principal_received: float (nullable = true)
 |-- total_interest_received: float (nullable = true)
 |-- total_late_fee_received: float (nullable = true)
 |-- total_payment_received: float (nullable = true)
 |-- last_payment_amount: float (nullable = true)
 |-- last_payment_date: string (nullable = true)
 |-- next_payment_date: string (nullable = true)



#### put a ingest date 

In [8]:
from pyspark.sql.functions import current_timestamp

In [9]:
loan_repayments_ingestd = loan_repayments_raw_df.withColumn("ingest_date",current_timestamp())

In [10]:
loan_repayments_ingestd

+-------+------------------------+-----------------------+-----------------------+----------------------+-------------------+-----------------+-----------------+--------------------+
|loan_id|total_principal_received|total_interest_received|total_late_fee_received|total_payment_received|last_payment_amount|last_payment_date|next_payment_date|         ingest_date|
+-------+------------------------+-----------------------+-----------------------+----------------------+-------------------+-----------------+-----------------+--------------------+
|6300587|                 12000.0|                1864.87|                  20.86|             13885.727|             108.83|         Nov-2014|             null|2025-06-17 22:43:...|
|6311418|                  7125.0|                  95.92|                    0.0|             7220.9243|            7220.52|         Sep-2013|             null|2025-06-17 22:43:...|
|6321535|                 24000.0|                7418.84|                    0.0|              31418.84|           13979.48|         Apr-2016|             null|2025-06-17 22:43:...|
|6300526|                 24000.0|                2296.28|                    0.0|             26296.275|             730.17|         Aug-2016|             null|2025-06-17 22:43:...|
|6301469|                 18000.0|                9239.79|                    0.0|             27239.795|             452.24|         Aug-2018|             null|2025-06-17 22:43:...|
|6306881|                 20000.0|                2469.32|                    0.0|             22469.322|           16744.64|         Oct-2014|             null|2025-06-17 22:43:...|
|6301579|                 12000.0|                2089.07|                    0.0|              14089.07|            4189.07|         Oct-2015|             null|2025-06-17 22:43:...|
|6291661|                 28000.0|               12441.42|                    0.0|              40441.42|           10660.91|         Apr-2017|             null|2025-06-17 22:43:...|
|6311612|                  3629.9|                 847.39|                    0.0|               4478.25|             179.13|         Sep-2015|             null|2025-06-17 22:43:...|
|6311704|                 23600.0|                 2077.3|                    0.0|             25677.297|           21445.23|         Apr-2014|             null|2025-06-17 22:43:...|
|6301559|                 13375.0|                3926.19|                    0.0|             17301.188|             480.18|         Aug-2016|             null|2025-06-17 22:43:...|
|6301412|                 17500.0|                2062.85|                    0.0|              19562.85|            3748.28|         Jan-2016|             null|2025-06-17 22:43:...|
|6331614|                 14000.0|                2442.47|                    0.0|              16442.47|            4430.47|         Nov-2015|             null|2025-06-17 22:43:...|
|6331589|                 1458.43|                1100.69|                    0.0|               4051.94|             426.57|         Mar-2014|             null|2025-06-17 22:43:...|
|6321730|                 14000.0|                 719.51|                    0.0|             14719.506|           11239.57|         May-2014|             null|2025-06-17 22:43:...|
|6321586|                 15000.0|                1929.76|                  25.29|              16955.05|           10354.66|         Oct-2014|             null|2025-06-17 22:43:...|
|6331533|                 10000.0|                2248.02|                    0.0|             12248.021|             339.97|         Aug-2016|             null|2025-06-17 22:43:...|
|6321666|                 6239.23|                1316.72|                    0.0|               7819.45|             260.55|         Jan-2016|             null|2025-06-17 22:43:...|
|6328050|                 10000.0|                2617.64|                    0.0|              12617.64|            4937.87|         Jun-2015|             n

In [11]:
loan_repayments_ingestd.createOrReplaceTempView("loan_repayments")

In [12]:
loan_repayments_ingestd.count()

2260701

In [13]:
spark.sql("select * from loan_repayments")

+-------+------------------------+-----------------------+-----------------------+----------------------+-------------------+-----------------+-----------------+--------------------+
|loan_id|total_principal_received|total_interest_received|total_late_fee_received|total_payment_received|last_payment_amount|last_payment_date|next_payment_date|         ingest_date|
+-------+------------------------+-----------------------+-----------------------+----------------------+-------------------+-----------------+-----------------+--------------------+
|6300587|                 12000.0|                1864.87|                  20.86|             13885.727|             108.83|         Nov-2014|             null|2025-06-17 22:43:...|
|6311418|                  7125.0|                  95.92|                    0.0|             7220.9243|            7220.52|         Sep-2013|             null|2025-06-17 22:43:...|
|6321535|                 24000.0|                7418.84|                    0.0|              31418.84|           13979.48|         Apr-2016|             null|2025-06-17 22:43:...|
|6300526|                 24000.0|                2296.28|                    0.0|             26296.275|             730.17|         Aug-2016|             null|2025-06-17 22:43:...|
|6301469|                 18000.0|                9239.79|                    0.0|             27239.795|             452.24|         Aug-2018|             null|2025-06-17 22:43:...|
|6306881|                 20000.0|                2469.32|                    0.0|             22469.322|           16744.64|         Oct-2014|             null|2025-06-17 22:43:...|
|6301579|                 12000.0|                2089.07|                    0.0|              14089.07|            4189.07|         Oct-2015|             null|2025-06-17 22:43:...|
|6291661|                 28000.0|               12441.42|                    0.0|              40441.42|           10660.91|         Apr-2017|             null|2025-06-17 22:43:...|
|6311612|                  3629.9|                 847.39|                    0.0|               4478.25|             179.13|         Sep-2015|             null|2025-06-17 22:43:...|
|6311704|                 23600.0|                 2077.3|                    0.0|             25677.297|           21445.23|         Apr-2014|             null|2025-06-17 22:43:...|
|6301559|                 13375.0|                3926.19|                    0.0|             17301.188|             480.18|         Aug-2016|             null|2025-06-17 22:43:...|
|6301412|                 17500.0|                2062.85|                    0.0|              19562.85|            3748.28|         Jan-2016|             null|2025-06-17 22:43:...|
|6331614|                 14000.0|                2442.47|                    0.0|              16442.47|            4430.47|         Nov-2015|             null|2025-06-17 22:43:...|
|6331589|                 1458.43|                1100.69|                    0.0|               4051.94|             426.57|         Mar-2014|             null|2025-06-17 22:43:...|
|6321730|                 14000.0|                 719.51|                    0.0|             14719.506|           11239.57|         May-2014|             null|2025-06-17 22:43:...|
|6321586|                 15000.0|                1929.76|                  25.29|              16955.05|           10354.66|         Oct-2014|             null|2025-06-17 22:43:...|
|6331533|                 10000.0|                2248.02|                    0.0|             12248.021|             339.97|         Aug-2016|             null|2025-06-17 22:43:...|
|6321666|                 6239.23|                1316.72|                    0.0|               7819.45|             260.55|         Jan-2016|             null|2025-06-17 22:43:...|
|6328050|                 10000.0|                2617.64|                    0.0|              12617.64|            4937.87|         Jun-2015|             n

#### let's check how many columns are null 


In [14]:
spark.sql("select count(*) from loan_repayments where total_principal_received IS NULL")

+--------+
|count(1)|
+--------+
|      69|
+--------+

#### if any of the numeric columns are null, i want to drop that row

In [15]:
columns_to_check = ['total_principal_received', 'total_interest_received', 'total_late_fee_received', 'total_payment_received', 'last_payment_amount']

In [16]:
loan_repayments_filtered = loan_repayments_ingestd.na.drop(subset=columns_to_check)

In [17]:
loan_repayments_filtered.count()

2260498

In [18]:
loan_repayments_filtered.createOrReplaceTempView("loan_repayments")

#### let's see if total_payment_received is 0 in any case

In [19]:
spark.sql("select * from loan_repayments where total_payment_received = 0.0")

+--------+------------------------+-----------------------+-----------------------+----------------------+-------------------+-----------------+-----------------+--------------------+
| loan_id|total_principal_received|total_interest_received|total_late_fee_received|total_payment_received|last_payment_amount|last_payment_date|next_payment_date|         ingest_date|
+--------+------------------------+-----------------------+-----------------------+----------------------+-------------------+-----------------+-----------------+--------------------+
| 5956295|                     0.0|                    0.0|                    0.0|                   0.0|                0.0|             null|             null|2025-06-17 22:43:...|
| 4975367|                     0.0|                    0.0|                    0.0|                   0.0|                0.0|             null|             null|2025-06-17 22:43:...|
| 5104779|                     0.0|                    0.0|                    0.0|                   0.0|                0.0|             null|             null|2025-06-17 22:43:...|
| 4724657|                     0.0|                    0.0|                    0.0|                   0.0|                0.0|             null|             null|2025-06-17 22:43:...|
| 3808675|                     0.0|                    0.0|                    0.0|                   0.0|                0.0|             null|             null|2025-06-17 22:43:...|
| 3706521|                     0.0|                    0.0|                    0.0|                   0.0|                0.0|             null|             null|2025-06-17 22:43:...|
| 3104822|                     0.0|                    0.0|                    0.0|                   0.0|                0.0|             null|             null|2025-06-17 22:43:...|
| 2310883|                     0.0|                    0.0|                    0.0|                   0.0|                0.0|             null|             null|2025-06-17 22:43:...|
| 1598556|                     0.0|                    0.0|                    0.0|                   0.0|                0.0|             null|             null|2025-06-17 22:43:...|
| 1598029|                     0.0|                    0.0|                    0.0|                   0.0|                0.0|             null|             null|2025-06-17 22:43:...|
| 1480469|                     0.0|                    0.0|                    0.0|                   0.0|                0.0|             null|             null|2025-06-17 22:43:...|
| 1263012|                     0.0|                    0.0|                    0.0|                   0.0|                0.0|             null|             null|2025-06-17 22:43:...|
| 1140520|                     0.0|                    0.0|                    0.0|                   0.0|                0.0|             null|             null|2025-06-17 22:43:...|
| 1136894|                     0.0|                    0.0|                    0.0|                   0.0|                0.0|             null|             null|2025-06-17 22:43:...|
| 1132481|                     0.0|                    0.0|                    0.0|                   0.0|                0.0|             null|             null|2025-06-17 22:43:...|
| 1108831|                     0.0|                    0.0|                    0.0|                   0.0|                0.0|             null|             null|2025-06-17 22:43:...|
| 1100834|                     0.0|                    0.0|                    0.0|                   0.0|                0.0|             null|             null|2025-06-17 22:43:...|
| 1082215|                     0.0|                    0.0|                    0.0|                   0.0|                0.0|             null|             null|2025-06-17 22:43:...|
| 1081948|                     0.0|                    0.0|                    0.0|                   0.0|                0.0|          

In [20]:
spark.sql("select count(*) from loan_repayments where total_payment_received = 0.0 and total_principal_received != 0.0")

+--------+
|count(1)|
+--------+
|      46|
+--------+

#### in such records, we can sum up total_principal, total_interest and total_late_fee and replace 0 with the result

In [21]:
from pyspark.sql.functions import when, col

#### withColumn is used to change the column
#### the column which I want to deal with is "total_payments_received"
#### on what basis
#### when column is not equal to 0- it's not empty
#### and total_payment_received is 0- that means something wrong is there
#### THIS IS THE CONDITION
#### if this condition is met
#### replace it with this value
#### otherwise, do not do anything

In [22]:
loans_payments_fixed_df = loan_repayments_filtered.withColumn(
   "total_payment_received",
    when(
        (col("total_principal_received") != 0.0) &
        (col("total_payment_received") == 0.0),
        col("total_principal_received") + col("total_interest_received") + col("total_late_fee_received")
    ).otherwise(col("total_payment_received"))
)

In [23]:
loans_payments_fixed_df

+-------+------------------------+-----------------------+-----------------------+----------------------+-------------------+-----------------+-----------------+--------------------+
|loan_id|total_principal_received|total_interest_received|total_late_fee_received|total_payment_received|last_payment_amount|last_payment_date|next_payment_date|         ingest_date|
+-------+------------------------+-----------------------+-----------------------+----------------------+-------------------+-----------------+-----------------+--------------------+
|6300587|                 12000.0|                1864.87|                  20.86|             13885.727|             108.83|         Nov-2014|             null|2025-06-17 22:43:...|
|6311418|                  7125.0|                  95.92|                    0.0|             7220.9243|            7220.52|         Sep-2013|             null|2025-06-17 22:43:...|
|6321535|                 24000.0|                7418.84|                    0.0|              31418.84|           13979.48|         Apr-2016|             null|2025-06-17 22:43:...|
|6300526|                 24000.0|                2296.28|                    0.0|             26296.275|             730.17|         Aug-2016|             null|2025-06-17 22:43:...|
|6301469|                 18000.0|                9239.79|                    0.0|             27239.795|             452.24|         Aug-2018|             null|2025-06-17 22:43:...|
|6306881|                 20000.0|                2469.32|                    0.0|             22469.322|           16744.64|         Oct-2014|             null|2025-06-17 22:43:...|
|6301579|                 12000.0|                2089.07|                    0.0|              14089.07|            4189.07|         Oct-2015|             null|2025-06-17 22:43:...|
|6291661|                 28000.0|               12441.42|                    0.0|              40441.42|           10660.91|         Apr-2017|             null|2025-06-17 22:43:...|
|6311612|                  3629.9|                 847.39|                    0.0|               4478.25|             179.13|         Sep-2015|             null|2025-06-17 22:43:...|
|6311704|                 23600.0|                 2077.3|                    0.0|             25677.297|           21445.23|         Apr-2014|             null|2025-06-17 22:43:...|
|6301559|                 13375.0|                3926.19|                    0.0|             17301.188|             480.18|         Aug-2016|             null|2025-06-17 22:43:...|
|6301412|                 17500.0|                2062.85|                    0.0|              19562.85|            3748.28|         Jan-2016|             null|2025-06-17 22:43:...|
|6331614|                 14000.0|                2442.47|                    0.0|              16442.47|            4430.47|         Nov-2015|             null|2025-06-17 22:43:...|
|6331589|                 1458.43|                1100.69|                    0.0|               4051.94|             426.57|         Mar-2014|             null|2025-06-17 22:43:...|
|6321730|                 14000.0|                 719.51|                    0.0|             14719.506|           11239.57|         May-2014|             null|2025-06-17 22:43:...|
|6321586|                 15000.0|                1929.76|                  25.29|              16955.05|           10354.66|         Oct-2014|             null|2025-06-17 22:43:...|
|6331533|                 10000.0|                2248.02|                    0.0|             12248.021|             339.97|         Aug-2016|             null|2025-06-17 22:43:...|
|6321666|                 6239.23|                1316.72|                    0.0|               7819.45|             260.55|         Jan-2016|             null|2025-06-17 22:43:...|
|6328050|                 10000.0|                2617.64|                    0.0|              12617.64|            4937.87|         Jun-2015|             n

#### now i want to remove the other rows where this value is 0.0


In [24]:
loans_payments_fixed_df.filter("total_payment_received = 0.0").count()


949

In [25]:
loan_repayments_fixed2 = loans_payments_fixed_df.filter("total_payment_received != 0.0")


In [26]:
loan_repayments_fixed2

+-------+------------------------+-----------------------+-----------------------+----------------------+-------------------+-----------------+-----------------+--------------------+
|loan_id|total_principal_received|total_interest_received|total_late_fee_received|total_payment_received|last_payment_amount|last_payment_date|next_payment_date|         ingest_date|
+-------+------------------------+-----------------------+-----------------------+----------------------+-------------------+-----------------+-----------------+--------------------+
|6300587|                 12000.0|                1864.87|                  20.86|             13885.727|             108.83|         Nov-2014|             null|2025-06-17 22:43:...|
|6311418|                  7125.0|                  95.92|                    0.0|             7220.9243|            7220.52|         Sep-2013|             null|2025-06-17 22:43:...|
|6321535|                 24000.0|                7418.84|                    0.0|              31418.84|           13979.48|         Apr-2016|             null|2025-06-17 22:43:...|
|6300526|                 24000.0|                2296.28|                    0.0|             26296.275|             730.17|         Aug-2016|             null|2025-06-17 22:43:...|
|6301469|                 18000.0|                9239.79|                    0.0|             27239.795|             452.24|         Aug-2018|             null|2025-06-17 22:43:...|
|6306881|                 20000.0|                2469.32|                    0.0|             22469.322|           16744.64|         Oct-2014|             null|2025-06-17 22:43:...|
|6301579|                 12000.0|                2089.07|                    0.0|              14089.07|            4189.07|         Oct-2015|             null|2025-06-17 22:43:...|
|6291661|                 28000.0|               12441.42|                    0.0|              40441.42|           10660.91|         Apr-2017|             null|2025-06-17 22:43:...|
|6311612|                  3629.9|                 847.39|                    0.0|               4478.25|             179.13|         Sep-2015|             null|2025-06-17 22:43:...|
|6311704|                 23600.0|                 2077.3|                    0.0|             25677.297|           21445.23|         Apr-2014|             null|2025-06-17 22:43:...|
|6301559|                 13375.0|                3926.19|                    0.0|             17301.188|             480.18|         Aug-2016|             null|2025-06-17 22:43:...|
|6301412|                 17500.0|                2062.85|                    0.0|              19562.85|            3748.28|         Jan-2016|             null|2025-06-17 22:43:...|
|6331614|                 14000.0|                2442.47|                    0.0|              16442.47|            4430.47|         Nov-2015|             null|2025-06-17 22:43:...|
|6331589|                 1458.43|                1100.69|                    0.0|               4051.94|             426.57|         Mar-2014|             null|2025-06-17 22:43:...|
|6321730|                 14000.0|                 719.51|                    0.0|             14719.506|           11239.57|         May-2014|             null|2025-06-17 22:43:...|
|6321586|                 15000.0|                1929.76|                  25.29|              16955.05|           10354.66|         Oct-2014|             null|2025-06-17 22:43:...|
|6331533|                 10000.0|                2248.02|                    0.0|             12248.021|             339.97|         Aug-2016|             null|2025-06-17 22:43:...|
|6321666|                 6239.23|                1316.72|                    0.0|               7819.45|             260.55|         Jan-2016|             null|2025-06-17 22:43:...|
|6328050|                 10000.0|                2617.64|                    0.0|              12617.64|            4937.87|         Jun-2015|             n

In [27]:
loan_repayments_fixed2.createOrReplaceTempView("loan_repayments")

In [28]:
loan_repayments_fixed2.filter("last_payment_date = 0.0").count()


48

In [29]:
loan_repayments_fixed2.filter("next_payment_date = 0.0").count()

24

#### we can use a when clause

In [32]:
loans_repayments_ldate_fixed = loan_repayments_fixed2.withColumn(
   "last_payment_date",
    when(
        (col("last_payment_date") == 0.0),
    None 
    ).otherwise(col("last_payment_date"))
)

In [37]:
loans_repayments_date_fixed = loan_repayments_fixed2.withColumn(
   "next_payment_date",
    when(
        (col("next_payment_date") == 0.0),
    None 
    ).otherwise(col("next_payment_date"))
)

In [38]:
loans_repayments_ldate_fixed

DataFrame[loan_id: string, total_principal_received: float, total_interest_received: float, total_late_fee_received: float, total_payment_received: float, last_payment_amount: float, last_payment_date: string, next_payment_date: string, ingest_date: timestamp]

In [36]:
loans_repayments_date_fixed

+-------+------------------------+-----------------------+-----------------------+----------------------+-------------------+-----------------+-----------------+--------------------+
|loan_id|total_principal_received|total_interest_received|total_late_fee_received|total_payment_received|last_payment_amount|last_payment_date|next_payment_date|         ingest_date|
+-------+------------------------+-----------------------+-----------------------+----------------------+-------------------+-----------------+-----------------+--------------------+
|6300587|                 12000.0|                1864.87|                  20.86|             13885.727|             108.83|         Nov-2014|             null|2025-06-17 22:44:...|
|6311418|                  7125.0|                  95.92|                    0.0|             7220.9243|            7220.52|         Sep-2013|             null|2025-06-17 22:44:...|
|6321535|                 24000.0|                7418.84|                    0.0|              31418.84|           13979.48|         Apr-2016|             null|2025-06-17 22:44:...|
|6300526|                 24000.0|                2296.28|                    0.0|             26296.275|             730.17|         Aug-2016|             null|2025-06-17 22:44:...|
|6301469|                 18000.0|                9239.79|                    0.0|             27239.795|             452.24|         Aug-2018|             null|2025-06-17 22:44:...|
|6306881|                 20000.0|                2469.32|                    0.0|             22469.322|           16744.64|         Oct-2014|             null|2025-06-17 22:44:...|
|6301579|                 12000.0|                2089.07|                    0.0|              14089.07|            4189.07|         Oct-2015|             null|2025-06-17 22:44:...|
|6291661|                 28000.0|               12441.42|                    0.0|              40441.42|           10660.91|         Apr-2017|             null|2025-06-17 22:44:...|
|6311612|                  3629.9|                 847.39|                    0.0|               4478.25|             179.13|         Sep-2015|             null|2025-06-17 22:44:...|
|6311704|                 23600.0|                 2077.3|                    0.0|             25677.297|           21445.23|         Apr-2014|             null|2025-06-17 22:44:...|
|6301559|                 13375.0|                3926.19|                    0.0|             17301.188|             480.18|         Aug-2016|             null|2025-06-17 22:44:...|
|6301412|                 17500.0|                2062.85|                    0.0|              19562.85|            3748.28|         Jan-2016|             null|2025-06-17 22:44:...|
|6331614|                 14000.0|                2442.47|                    0.0|              16442.47|            4430.47|         Nov-2015|             null|2025-06-17 22:44:...|
|6331589|                 1458.43|                1100.69|                    0.0|               4051.94|             426.57|         Mar-2014|             null|2025-06-17 22:44:...|
|6321730|                 14000.0|                 719.51|                    0.0|             14719.506|           11239.57|         May-2014|             null|2025-06-17 22:44:...|
|6321586|                 15000.0|                1929.76|                  25.29|              16955.05|           10354.66|         Oct-2014|             null|2025-06-17 22:44:...|
|6331533|                 10000.0|                2248.02|                    0.0|             12248.021|             339.97|         Aug-2016|             null|2025-06-17 22:44:...|
|6321666|                 6239.23|                1316.72|                    0.0|               7819.45|             260.55|         Jan-2016|             null|2025-06-17 22:44:...|
|6328050|                 10000.0|                2617.64|                    0.0|              12617.64|            4937.87|         Jun-2015|             n

In [39]:
loans_repayments_date_fixed.createOrReplaceTempView("loan_repayments")

#### Put it to cleaned folder

In [40]:
loans_repayments_date_fixed.write \
.format("csv") \
.option("header",True) \
.mode("overwrite") \
.option("path","/user/itv018355/lendingclubproject/cleaned/loan_repayments_csv") \
.save()

In [41]:
loans_repayments_date_fixed.write \
.format("parquet") \
.option("header",True) \
.mode("overwrite") \
.option("path","/user/itv018355/lendingclubproject/cleaned/loan_repayments_parquet") \
.save()